In [ ]:
import pandas as pd
import seaborn as sns
from utils import get_results

In [ ]:
def headways(trips, rt, direction):
    tr = trips[(trips['stop_sequence']==1) & 
               (trips['route_id']==rt) & 
               (trips['direction']==direction)].copy()
    tr = tr.sort_values(by='departure_time_sec')
    tr['headway'] = tr['departure_time_sec'].diff()
    tr['schd_headway'] = tr['schd_time_sec'].diff()
    tr['schd_time'] = pd.to_timedelta(tr['schd_time_sec'], unit='S')
    tr['arr_time'] = pd.to_timedelta(tr['arrival_time_sec'].round(), unit='S')
    tr['dep_time'] = pd.to_timedelta(tr['departure_time_sec'].round(), unit='S')
    return tr[['stop_id', 'trip_id', 'run_id', 'route_id', 
               'block_id', 'headway', 'schd_headway',
               'arr_time', 'dep_time', 'schd_time']]

def barplots(r):
    r['wait_time'] = r['wait_time'].round("S")
    r['wait_time_mins']= r['wait_time'].dt.total_seconds()/60
    r['rt_dir'] = r['route_id'].astype(str) + '-' +r['direction']
    sns.barplot(r, y='wait_time_mins', x='rt_dir', hue='scenario')
    # plt.ylim(0,20)

In [ ]:
pax = pd.read_csv('data/sim_out/experiments_0611-2304/pax.csv')

trips = pd.read_csv('data/sim_out/experiments_0611-2304/trips.csv')
for ky in ('arrival', 'departure', 'schd'):
    full_ky = ky + '_time_sec'
    trips[full_ky] = pd.to_datetime(trips[full_ky]).dt.round("S")

for ky in ('arrival', 'alighting', 'boarding'):
    full_ky = ky + '_time'
    pax[full_ky] = pd.to_datetime(pax[full_ky]).dt.round("S")

In [ ]:
am_pax = pax[pax['arrival_time'].dt.hour.isin([6,7,8])].copy()
am_pax['wait_time'] = am_pax['boarding_time'] - am_pax['arrival_time']
am_pax['date'] = am_pax['arrival_time'].dt.date

In [ ]:
results = am_pax.groupby(
    ['scenario','route_id', 'direction'])['wait_time'].mean().reset_index()
results_95 = am_pax.groupby(
    ['scenario','route_id', 'direction'])['wait_time'].quantile(0.9).reset_index()

In [ ]:
barplots(results)

In [ ]:
barplots(results_95)

In [ ]:
tri1 = pd.read_csv('data/sim_out/experiments_0413-1258/trips.csv')
# tri2 = pd.read_csv('data/sim_out/experiments_0413-1258/EHD/trips.csv')

In [ ]:
hws = headways(tri2, 91, 'South')
hws['arr_time'] += pd.to_datetime('2022-11-15')
hws

In [ ]:
start_time = 7*60*60
end_time = 9*60*60
res = get_results('0529-1244', ['EHD', 'NC'], start_time, end_time)
res

In [ ]:
def wait_t(df, st, et):
    wdf = df[(df['arrival_time'] > st) & (df['arrival_time'] < et)].copy()
    wdf['wait'] = wdf['boarding_time'] - wdf['arrival_time']
    return wdf[['boarding_stop', 'alighting_stop', 'route_id', 'wait', 'direction']]

In [ ]:
pax1 = pd.read_csv('data/sim_out/experiments_0607-0053/NC/pax.csv')
pax2 = pd.read_csv('data/sim_out/experiments_0607-0053/EHD/pax.csv')

arr0 = 20300
arr1 = 30000

w1 = wait_t(pax1, arr0, arr1)
w2 = wait_t(pax2, arr0, arr1)

w1 = w1.groupby(['route_id', 'direction'])['wait'].mean().reset_index()
w2 = w2.groupby(['route_id','direction'])['wait'].mean().reset_index()

for w, k in zip([w1, w2], ['DN', 'EHD']):
    w['rt_dir'] = w['route_id'].astype(str) + '-' + w['direction']
    w['scenario'] = k

results = pd.concat([w1, w2], ignore_index=True)

sns.barplot(data=results, hue='scenario', y='wait', x='rt_dir')